# Test 3DVar and 4DVar implementation

In [1]:
import torch
from deepda import apply_3DVar, apply_4DVar, forwardModel_r
from math import ceil

In [2]:
def H(x: torch.Tensor):
    return x

In [3]:
B = torch.eye(3, dtype=float)
R = torch.eye(3, dtype=float)
y = torch.tensor([10, 20, 30], dtype=float)
xb = torch.zeros_like(y)

In [4]:
apply_3DVar(H, B, R, xb, y, learning_rate=2)

Iterations: 0, Norm of J gradient: 74.83314773547883
Iterations: 1, Norm of J gradient: 62.22539674733033
Iterations: 2, Norm of J gradient: 50.39654349988177
Iterations: 3, Norm of J gradient: 39.80768151544453
Iterations: 4, Norm of J gradient: 30.81037066896585
Iterations: 5, Norm of J gradient: 23.481934668704795
Iterations: 6, Norm of J gradient: 18.00583590323266
Iterations: 7, Norm of J gradient: 14.98685068159499
Iterations: 8, Norm of J gradient: 14.75896581795235
Iterations: 9, Norm of J gradient: 16.46552873316387
Iterations: 10, Norm of J gradient: 18.841024238307245
Iterations: 11, Norm of J gradient: 21.07630524793385
Iterations: 12, Norm of J gradient: 22.751000530206905
Iterations: 13, Norm of J gradient: 23.645173143473976
Iterations: 14, Norm of J gradient: 23.674485193715306
Iterations: 15, Norm of J gradient: 22.87781359777211
Iterations: 16, Norm of J gradient: 21.40020602582196
Iterations: 17, Norm of J gradient: 19.463549549233882
Iterations: 18, Norm of J gradie

tensor([ 5.0000, 10.0000, 15.0000], dtype=torch.float64)

In [5]:
# We define the control parameters here
rayleigh = 35
prandtl = 10.
b = 8./3.
# rayleigh = 0.
# prandtl = 0.
# b = 0.
# initial condition for the true reference trajectory
x0 = torch.tensor([0., 1., 2.], dtype=float)

# integration time parameter
dt = 1.e-3      # This is time step size
T = 2.         # Total integration time, can be as short as 10 to speed things up
n_steps = ceil(T / dt)
time = torch.linspace(0., T, n_steps + 1)  # array of discrete times

# numerical integration given initial conditions and control parameters
xt = forwardModel_r(x0, time, rayleigh, prandtl, b)

In [6]:
# Which variables do we observe?
WhichVariablesAreObserved = torch.tensor([1, 1, 1], dtype=float)
#  Determines which variables are available to
#  the EnKF. For example:
#  WhichVariablesAreObserved = [1 1 1];
#  means: X, Y, Z are observed
#  WhichVariablesAreObserved = [1 0 1];
#  means: X and Z are observed
#  WhichVariablesAreObserved = [1 0 0];
# means: X is observed
sigobs = 2.  # standard deviation of the observation noise
# We generate the synthetic data
#  Construct observation matrix H
#  ........................................................................
# H = torch.diag(WhichVariablesAreObserved)
y_size = int(WhichVariablesAreObserved.sum())
# How often do we observe the true state?
dtobs = 0.5  # time between observations
nobs = ceil(T / dtobs) - 1  # number of times observations are performed
# no observation at t=0
gap = int(dtobs / dt)  # number of time steps between each observation
time_obs = time[gap::gap]
# Generate vector of observations
y = torch.zeros((y_size, nobs), dtype=float)
R = torch.diag(torch.tile(torch.tensor(sigobs**2, dtype=float), (y_size,)))
sqrt_s = torch.sqrt(R)
# y = Hxt
y = H(xt[:, gap::gap])
# compute observation error
noise = sqrt_s @ torch.randn(size=y.shape, dtype=float)
# y = Hxt + epsilon
y = y + noise

In [7]:
apply_4DVar(nobs, time_obs, gap, forwardModel_r, H, B, R, x0, y, model_args=(rayleigh, prandtl, b), learning_rate=0.0125)

Iterations: 0, Norm of J gradient: 1957.415551609941
Iterations: 1, Norm of J gradient: 1786.964204571218
Iterations: 2, Norm of J gradient: 1633.015051425685
Iterations: 3, Norm of J gradient: 1493.985311843024
Iterations: 4, Norm of J gradient: 1368.4135547859948
Iterations: 5, Norm of J gradient: 1254.9599450388125
Iterations: 6, Norm of J gradient: 1152.4033671702796
Iterations: 7, Norm of J gradient: 1059.6363546696819
Iterations: 8, Norm of J gradient: 975.6585731080709
Iterations: 9, Norm of J gradient: 899.5694463079931
Iterations: 10, Norm of J gradient: 830.5603762559236
Iterations: 11, Norm of J gradient: 767.9068913550376
Iterations: 12, Norm of J gradient: 710.960962589866
Iterations: 13, Norm of J gradient: 659.1436513200168
Iterations: 14, Norm of J gradient: 611.9381933107476
Iterations: 15, Norm of J gradient: 568.8835786646825
Iterations: 16, Norm of J gradient: 529.5686540358447
Iterations: 17, Norm of J gradient: 493.6267495917332
Iterations: 18, Norm of J gradient:

tensor([ 0.1265,  1.2907, -5.3889], dtype=torch.float64)